<a href="https://colab.research.google.com/github/umututku03/3D-Rendering-Camera-Rotation/blob/main/3D_Rendering_Camera_Rot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image, ImageDraw
import random
from math import sin, cos, radians

In [ ]:
# Define the cube vertices and edges
cube_vertices = np.array([
    [1, 1, -1], [-1, 1, -1], [-1, -1, -1], [1, -1, -1],
    [1, 1, 1], [-1, 1, 1], [-1, -1, 1], [1, -1, 1]
])

cube_edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),
    (4, 5), (5, 6), (6, 7), (7, 4),
    (0, 4), (1, 5), (2, 6), (3, 7)
]

In [ ]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

In [ ]:
def look_at(from_pos, to_pos, up):
    forward = normalize(from_pos - to_pos)
    right = normalize(np.cross(up, forward))
    up = np.cross(forward, right)
    view_matrix = np.array([
        [right[0], right[1], right[2], -np.dot(right, from_pos)],
        [up[0], up[1], up[2], -np.dot(up, from_pos)],
        [forward[0], forward[1], forward[2], -np.dot(forward, from_pos)],
        [0, 0, 0, 1]
    ])
    return view_matrix

In [ ]:
def project_vertex(vertex, view_matrix, projection_matrix):
    vertex_homogeneous = np.append(vertex, 1)
    transformed_vertex = view_matrix.dot(vertex_homogeneous)
    projected_vertex = projection_matrix.dot(transformed_vertex)
    return projected_vertex[:2] / projected_vertex[3]

In [ ]:
# Initial camera setup
initial_camera_pos = np.array([7.0, 1.0, 1.0])
target_pos = np.array([0.0, 0.0, 0.0])
up_vector = np.array([0.0, 1.0, 0.0])

In [ ]:
# Projection matrix setup (assuming perspective projection)
fov = 60  # Field of view in degrees - could make it bigger
aspect_ratio = 1.0  # Aspect ratio
near = 1.0  # Near clipping plane
far = 100.0  # Far clipping plane
fov_rad = np.radians(fov)
f = 1 / np.tan(fov_rad / 2)
projection_matrix = np.array([
    [f / aspect_ratio, 0, 0, 0],
    [0, f, 0, 0],
    [0, 0, (far + near) / (near - far), (2 * far * near) / (near - far)],
    [0, 0, -1, 0]
])

In [ ]:
# Rendering parameters
r = 5
num_steps = 100

In [ ]:
# Ensure the output directories exist
render_dir = "renders_pillow"
extrinsic_dir = "extrinsics"
points_dir = "points_2d"

for directory in [render_dir, extrinsic_dir, points_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
# Image rendering loop
for step in range(num_steps):
    alpha = step * 2 * np.pi / num_steps
    camera_pos = np.array([
        target_pos[0] + r * np.cos(alpha),  # x-coordinate
        target_pos[1] - 3.5,                # y-coordinate (elevation) kept constant
        target_pos[2] + r * np.sin(alpha)   # z-coordinate
    ])
    view_matrix = look_at(camera_pos, target_pos, up_vector)

    # Save the extrinsic matrix
    np.savetxt(f"{extrinsic_dir}/extrinsic_{step}.txt", view_matrix)

    image_size = 500
    image = Image.new("RGB", (image_size, image_size), "white")
    draw = ImageDraw.Draw(image)

    # Project and draw each edge of the cube
    points_2d = []
    for edge in cube_edges:
        v0, v1 = cube_vertices[edge[0]], cube_vertices[edge[1]]
        p0 = project_vertex(v0, view_matrix, projection_matrix)
        p1 = project_vertex(v1, view_matrix, projection_matrix)
        p0 = (image_size / 2 * (p0 + 1)).astype(int)
        p1 = (image_size / 2 * (p1 + 1)).astype(int)
        draw.line([tuple(p0), tuple(p1)], fill="black")
        points_2d.append(p0)
        points_2d.append(p1)

    # Save the 2D points
    points_2d = np.unique(np.array(points_2d), axis=0)  # Remove duplicate points
    np.savetxt(f"{points_dir}/points_{step}.txt", points_2d, fmt='%d')

    # Save the image
    filename = f'{render_dir}/frame_{step}.png'
    image.save(filename)

print(f'Rendering completed. Images saved in {render_dir}, {extrinsic_dir}, and {points_dir} directories.')

In [ ]:
# Ensure the output directories exist in the current working directory
# NOTE: This project is created in "Kaggle"
output_dir_frames = "/kaggle/working/cube_images_fixed_frames"
output_dir_extrinsics = "/kaggle/working/cube_images_fixed_extrinsics"
output_dir_points = "/kaggle/working/cube_images_fixed_points"

for directory in [output_dir_frames, output_dir_extrinsics, output_dir_points]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Copy images and data to the output directories
for file in os.listdir(render_dir):
    shutil.copy(os.path.join(render_dir, file), output_dir_frames)

for file in os.listdir(extrinsic_dir):
    shutil.copy(os.path.join(extrinsic_dir, file), output_dir_extrinsics)

for file in os.listdir(points_dir):
    shutil.copy(os.path.join(points_dir, file), output_dir_points)

# Create zip files
shutil.make_archive(output_dir_frames, 'zip', output_dir_frames)
shutil.make_archive(output_dir_extrinsics, 'zip', output_dir_extrinsics)
shutil.make_archive(output_dir_points, 'zip', output_dir_points)

print(f'Images and data are saved and zipped in {output_dir_frames}.zip, {output_dir_extrinsics}.zip, and {output_dir_points}.zip')